# Clasificación: Prediciendo sentimiento de reviews de productos

<img src='https://dealnews.a.ssl.fastly.net/files/uploads/funny-amazon-reviews-horse-head-mask-3.png'>

## Dataset
'amazon_baby.csv' contiene información de reviews de productos de bebe en Amazon. El propósito de este caso de estudio es construir un clasificador de sentimiento que pueda predecir si el review de un producto es positivo o negativo. 

Dentro del dataset se tiene los siguientes datos:
1. `review`: Texto del review escrito por un usuario
2. `name`: Nombre del producto
3. `rating`: Rating del 1 al 5
Existen muchas técnicas para el análisis de sentimiento en texto hoy en día, pero para efectos de este caso, el análisis de sentimiento lo realizaremos usando un conteo de palabras. 

Por ejemplo: Para el review "the sushi was good and the service was excellent" se generaría el conteo de palabras:
"the": 2
"sushi": 1
"was": 2
"good": 1
"and" 1
"service": 1
"excelente": 1

1. Usa `CountVectorize`que se encuentra en klearn.feature_extraction.text para obtener lo feature para tu modelo
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

2. Ahora usa `TfidfVectorizer` para obtgener los features para tu modelo y compáralo contra el anterior
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [26]:
import numpy as np 
import pandas as pd 

In [27]:
df = pd.read_csv('amazon_baby.csv')
df.shape

(183531, 3)

## Cleaning

In [28]:
df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


Para el própostio de este laboratorio, nos enfocaremos en la columna **review** y **rating**.
  
Empecemos por limpiar el dataframe, borrando todas las rows con datos nulos y limpiamos de las rows todo aquello que no sea letra o dígitos.  
La columna **rating**, va del 1 al 5, y en varios ejemplos eliminan las columnas con rating de 3, porque sería como un sentimiento neutral y no ayuda a la creación del modelo. Entonces agregaremos una columna **"Positivity"**, donde los ratings mayores a 3 se les asigna un valor de **1**, indicando que el sentimiento es **positivo**. De otra forma será **0**, indicando un sentimiento **negativo**. 



In [29]:
df = df.fillna({'review':''})  # fill in N/A's in the review column
import string
df['review']=df['review'].str.replace('[{}]'.format(string.punctuation), '')

df.dropna(inplace=True)
#df = df[pd.notnull(df['review'])]
df = df[df['rating'] != 3]
df['Positivity'] = np.where(df['rating'] > 3, 1, 0)
df.shape

(166456, 4)

In [30]:
df.head()

,name,review,rating,Positivity
1,Planetwise Wipe Pouch,it came early and was not disappointed i love ...,5,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase I h...,5,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried nonstop when I tried...,5,1
5,Stop Pacifier Sucking without tears with Thumb...,When the Binky Fairy came to our house we didn...,5,1


Ahora, le hacemos split a la data en un substet para un **training set** y un **test set**, usando **review** y **Positivity**. El split será de 80-20, e imprimimos la primera review para fines didacticos y el shape del training y el test. 

## Split training and test

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['Positivity'],test_size = 0.20, random_state = 1)
print('X_train first entry: \n\n', X_train.iloc[0])
print('\n\nX_train shape: ', X_train.shape)
print('\n\nX_test shape: ', X_test.shape)


X_train first entry: 

 I am editing this review since I have now used this item for a yearProsI liked the colors i chose they were as depicted when i was shopping for it  i bought white n beigeCons i find that the wood is painted poorly you can see stroke like residue and seems that it needed another hand or soIt looks kind of sloppy as well some bloches of paint In my opinion it is kind of expensive when it comes to the condition you  get the wood inIt makes a loud cracking noise every time I rock my baby to sleepThis is very infuriating when trying to get a baby to sleepOne of the iron springs keeps popping out and I need my husband to put it back in for meI am 115 lb so this has nothing to do with weightOverall if you are going to use it just to relax in it is fineIf you are trying to rock your baby to sleep forget it


X_train shape:  (133164,)


X_test shape:  (33292,)


En el **X_train**, podemos ver que tenemos una colección de más 133 mil reviews y más de 33 mil para **test**. Para poder aplicar machine learning a estos reviews o "documentos de texto", tenemos que convertir el contenido del texto en vectores de caracteríosticas númericas para usar con Scikit-Learn.

## Modelos
  
### CountVecotorizer-Bags-of-words
  
La manera más fácil de hacer esto, es usando la representación **bags-of-words**, que ignora la estructura y simplemente cuenta cuántas veces aparece cada palabra. **CountVectorizer**, nos permite usar el approach de bags-of-words al convertir una colección de documentos de texto en una matrix de recuentos de tokens. 

Para entenderlo mejor, veamos una representación gráfica de una Bag of words:

<img src='https://www.novuslight.com/uploads/n/BagofWords.jpg'>

Ok, quizás sea necesaria una mejpor representación gráfica:

<img src='http://images4.programmersought.com/947/0a/0acb9279d17a1631bcfb154583cca443.JPEG'>

  
Instanciamos el CountVectorizer y lo ajustamos a nuestra training data, conviertiendo nuestra colección de textos en una matriz de recuentos de tokens.

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer().fit(X_train)
vector

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

Este modelo tiene muchos paramentros, pero los valores por default nos sirven así como están. 

La configuración predeterminada tokeniza la cadena, extrayendo palabras de al menos 2 letras o números, separadas por límites de palabras, luego convierte todo en minúsculas y construye un vocabulario usando estos tokens. Podemos obtener algunos de los vocabularios usando el método **get_feature_names** de esta manera:

In [33]:
vector.get_feature_names()[1000::5000]

['140',
 '5hrs',
 'anywherefeaturesthe',
 'beingeasily',
 'byfisherprice',
 'colorsstyles',
 'daughgter',
 'dropscomes',
 'fangod',
 'gdiapers',
 'herespend',
 'iraqup',
 'limitssafe',
 'misgivings',
 'nosefrida',
 'painsoreness',
 'pregnet',
 'reducer',
 'seatjump',
 'snugsecure',
 'stylistic',
 'threes',
 'unfortuantely',
 'wellrecently']

Al observar esos vocabularios, podemos tener una pequeña idea de lo que tratan. Al verificar la longitud de get_feature_names, podemos ver que estamos trabajando con 119528 vocablos.

In [34]:
len(vector.get_feature_names())

119528

A continuación, transformamos los documentos en **X_train** en una "term matrix", lo que nos da la representación de bags-of-word. El resultado se almacena en una sparse matrix de SciPy, donde cada row corresponde a un documento, y cada columna es una palabra de nuestro vocabulario de training.

In [35]:
X_train_vectorized = vector.transform(X_train)
X_train_vectorized

<133164x119528 sparse matrix of type '<class 'numpy.int64'>'
	with 7025498 stored elements in Compressed Sparse Row format>

Las columnas quedan representadas de la siguiente forma:

<img src='https://www.oreilly.com/library/view/feature-engineering-for/9781491953235/assets/feml_0405.png'>

In [36]:
X_train_vectorized.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Las entradas en esta matriz son el número de veces que cada palabra aparece en cada review. Ya que el número de palabras en el vocabulario es más grande que el número de palabras que pueden aparecer en una sola review, la mayoría de los elementos en la matriz son cero. 
  

#### Logistic Regression  
  
Ahora, entrenaremos al clasificador de regresión logística, basado en matrix **X_train_vectorized**.

In [37]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
len(vector.get_feature_names())

/Users/jaimesolis/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jaimesolis/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


119528

#### Evaluate Model

Ahora, haremos predicciones usando X_test y calculando el score del area bajo la curva(probabilidad de que una palabra esté del lado correcto), la confusion matrix y sacando la precisión de nuestro modelo. 

In [38]:
from sklearn.metrics import roc_auc_score
predictions = model.predict(vector.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

from sklearn.metrics import confusion_matrix
print('\nConfusion Matrix: \n', confusion_matrix(y_test, predictions))

from sklearn.metrics import precision_score, recall_score
print("\nPrecision:", precision_score(y_test, predictions))
print("Recall:",recall_score(y_test, predictions))

AUC:  0.8500912650136809

Confusion Matrix: 
 [[ 3873  1429]
 [  848 27142]]

Precision: 0.9499842497637464
Recall: 0.9697034655234013


De la matriz de confusión podemos concluir que:  
**True positive: 3873**(Predijimos valores positivos y fueron positivos)  
**True negative: 27142**(Predijimos valores negativos y fueron negativos)  
**False positive: 1429**(Predijimos valores positivos y fueron negativos)  
**False negative: 848**(Predijimos valores negativos y fueron positivos)  

**Accuracy = (TP+TN)/total**  
**Accuracy = (3873+27142)/33292 ~ 93%**  
**Error Rate = (FP+FN)/total**  
**Error rate = (1429+848)/33292 ~7%**

El resultado no es malo. Para comprender mejor cómo nuestro modelo hace estas predicciones, podemos usar los coeficientes para cada característica (una palabra) para determinar su peso en términos de positividad y negatividad.

In [39]:
feature_names = np.array(vector.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()
print('Smallest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:15]]))
print('Largest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:-15:-1]]))

Smallest Coefs: 
['dissapointed' 'worst' 'useless' 'worthless' 'poorly' 'disappointing'
 'poor' 'unusable' 'theory' 'ineffective' 'concept' 'pointless'
 'unacceptable' 'returning' 'tomorrow']

Largest Coefs: 
['lifesaver' 'saves' 'ply' 'downside' 'thankful' 'worry' 'awesome'
 'relieved' 'glad' 'outstanding' 'skeptical' 'con' 'rich' 'minor']



Al ordenar los diez coeficientes más pequeños y diez más grandes, podemos ver que el modelo ha pronosticado palabras como “worst”, “disappointing” and “useless”" a críticas negativas, y palabras como “lifesaver”, “glad”, and “oustanding” a reviews positivos.

#### Test the model:

In [40]:
print(model.predict(vector.transform(['The product is not good, I would never buy them again',
                                    'The product is not bad, I will buy them again'])))

[1 0]


Nuestro modelo actual clasificó erróneamente el documento “The product is not good, I will never buy them again” como una review positiva, y también clasificó erróneamente el documento "“The product is not bad, I will buy them again” como una review negativa.


Sin embargo, nuestro modelo se puede mejorar. 

### Tf–idf term weighting  

En reviews largos, algunas palabras estarán presentes muy a menudo pero llevarán muy poca información significativa sobre el contenido real del documento (como  “the”, “a” and “is”). Si tuviéramos que alimentar los datos de conteo directamente a un clasificador, esos términos muy frecuentes opacarían las frecuencias de términos más raros aún más interesantes. **Tf-idf** nos permite ponderar los términos en función de lo **importantes** que son para un documento.
  
Por lo tanto, crearemos una instancia del Tf–idf vectorizer y lo ajustaremos a nuestros datos de entrenamiento. Especificamos min_df = 5, que eliminará cualquier palabra de nuestro vocabulario que **aparezca** en menos de cinco documentos.  

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer(min_df = 5).fit(X_train)
len(vector.get_feature_names())

20039

#### Logistic Regression  

In [42]:
X_train_vectorized = vector.transform(X_train)
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

/Users/jaimesolis/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

#### Evaluate Model

In [43]:
predictions = model.predict(vector.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

from sklearn.metrics import confusion_matrix
print('\nConfusion Matrix: \n', confusion_matrix(y_test, predictions))

from sklearn.metrics import precision_score, recall_score

print("\nPrecision:", precision_score(y_test, predictions))
print("Recall:",recall_score(y_test, predictions))

AUC:  0.833365313823213

Confusion Matrix: 
 [[ 3639  1663]
 [  549 27441]]

Precision: 0.9428600879604178
Recall: 0.9803858520900322


De la matriz de confusión podemos concluir que:  
**True positive: 3639**(Predijimos valores positivos y fueron positivos)  
**True negative: 27441**(Predijimos valores negativos y fueron negativos)  
**False positive: 1663**(Predijimos valores positivos y fueron negativos)  
**False negative: 549**(Predijimos valores negativos y fueron positivos)  

**Accuracy = (TP+TN)/total**  
**Accuracy = (3639+27441)/33292 ~ 93%**  
**Error Rate = (FP+FN)/total**  
**Error rate = (1663+549)/33292 ~7%**

Entonces, aunque pudimos reducir la cantidad de funciones de 119528 a solo 20039, nuestro AUC y Precisión se redujeron minimamente. 

Con el siguiente código, podemos obtener una lista de características con el tf-idf más pequeño que `aparece comúnmente en todas las revisiones o que rara vez aparece en reviews muy largas` y una lista de palabras con el tf-idf más grande contiene `palabras que aparecieron con frecuencia en una review, pero no aparecieron comúnmente en todas las revisiones`.

In [44]:
feature_names = np.array(vector.get_feature_names())
sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
print('Smallest Tfidf: \n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest Tfidf: \n{}\n'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Smallest Tfidf: 
['reviewfirst' 'situationthese' 'navigation' 'lockoffs' 'monthspros'
 'definitive' 'ubiquitous' 'goso' 'topmost' 'modelthe']

Largest Tfidf: 
['uacutetil' 'product' 'so' 'dd' 'excellent' 'excelente' 'excelent' 'love'
 'adorable' 'fab']



#### Test the model:

In [45]:
print(model.predict(vector.transform(['The product is not good, I will never buy them again',
                                    'The product is not bad, I will buy them again'])))

[0 0]


Nuestro modelo actual clasificó el documento “The product is not good, I will never buy them again” como una review negativa, pero también clasificó erróneamente el documento "“The product is not bad, I will buy them again” como una review negativa.

## n-grams  
  
Una forma de corregir esta clasificación errónea es agregar **n-grams**. Por ejemplo, los bigrams cuentan pares de palabras adyacentes, y podrían darnos características como "bad" o "not bad". Por lo tanto, estamos reajustando nuestro training especificando un min_df de 5 y extrayendo **1-grams** y **2-grams**.

In [46]:
vector = CountVectorizer(min_df = 5, ngram_range = (1,2)).fit(X_train)
X_train_vectorized = vector.transform(X_train)
len(vector.get_feature_names())

197286

#### Logistic Regresion

In [47]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

/Users/jaimesolis/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

#### Evaluate Model

Ahora tenemeos más caracteristicas para el training, veamos si el score aumenta. 

In [48]:
predictions = model.predict(vector.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

from sklearn.metrics import confusion_matrix
print('\nConfusion Matrix: \n', confusion_matrix(y_test, predictions))

from sklearn.metrics import precision_score, recall_score

print("Precision:", precision_score(y_test, predictions))
print("Recall:",recall_score(y_test, predictions))

AUC:  0.8877920578144723

Confusion Matrix: 
 [[ 4233  1069]
 [  638 27352]]
Precision: 0.9623869673832729
Recall: 0.9772061450518043


De la matriz de confusión podemos concluir que:  
**True positive: 3873**(Predijimos valores positivos y fueron positivos)  
**True negative: 27142**(Predijimos valores negativos y fueron negativos)  
**False positive: 1429**(Predijimos valores positivos y fueron negativos)  
**False negative: 848**(Predijimos valores negativos y fueron positivos)  

**Accuracy = (TP+TN)/total**  
**Accuracy = (3873+27142)/33292 ~ 93%**  
**Error Rate = (FP+FN)/total**  
**Error rate = (1429+848)/33292 ~7%**

Usando los coeficientes vamos a ver las caracteristicas con menor y mayor coeficiente: 

In [49]:
feature_names = np.array(vector.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()
print('Smallest Coef: \n{}\n'.format(feature_names[sorted_coef_index][:10]))
print('Largest Coef: \n{}\n'.format(feature_names[sorted_coef_index][:-11:-1]))

Smallest Coef: 
['not recommend' 'not worth' 'two stars' 'not happy' 'useless'
 'disappointing' 'poor' 'returned' 'disappointed' 'wouldn recommend']

Largest Coef: 
['not too' 'perfect' 'awesome' 'just what' 'excellent' 'these work' 'glad'
 'great' 'love' 'perfectly']



Nuestro modelo ha predicho correctamente bigrams como “not recommend”, “not worth” a reviews negativas, y bigrams como “these work”, “excellent” como reviews positivas.   
  
#### Test the model:

In [50]:
print(model.predict(vector.transform(['The product is not good, I would never buy them again',
                                    'The product is not bad, I will buy them again', 
                                      'This shit is awesome',
                                     'This shit doesnt work bad', 
                                     'this is totally a piece of shit'])))

[0 1 1 0 1]


Este último modelo identifica correctamente las reviews como negativas y positivas. 

<img src='https://media2.giphy.com/media/lTpme2Po0hkqI/giphy.gif'>